In [ ]:
!pip install pypulseq==1.3.1.post1 &> /dev/null
!pip install MRzeroCore &> /dev/null
!wget https://github.com/MRsources/MRzero-Core/raw/main/documentation/playground_mr0/subject05.npz &> /dev/null

In [ ]:
import numpy as np
# newer numpy versions don't contain this, but pypulseq still relies on it
np.int = int
np.float = float
np.complex = complex

import pypulseq as pp
import MRzeroCore as mr0
import torch
import matplotlib.pyplot as plt

experiment_id = "flash"

(pulseq_flash)=
# Pulseq and MRzeroCore

MRzero Core has functionality to parse and simulate pulseq .seq files.
We build the same pulseq sequence as before, but this time with pulseq.

In [ ]:
sys = pp.Opts(
    max_grad=28, grad_unit='mT/m', max_slew=150, slew_unit='T/m/s',
    rf_ringdown_time=20e-6, rf_dead_time=100e-6, adc_dead_time=20e-6,
    grad_raster_time=50e-6
)

In [ ]:
n_read = 64
n_phase = 64
fov = 192e-3
slice_thickness = 8e-3

rf = pp.make_sinc_pulse(
    flip_angle=5 * np.pi/180, duration=1e-3,
    slice_thickness=slice_thickness, apodization=0.5, time_bw_product=4,
    system=sys, return_gz=False
)
# Readout gradient
gx = pp.make_trapezoid('x', flat_area=n_read / fov, flat_time=n_read*50e-6, system=sys)
adc = pp.make_adc(
    num_samples=n_read, dwell=50e-6, delay=gx.rise_time,
    system=sys
)
# Rewinder before gx and spoiler afterwards
gx_pre = pp.make_trapezoid('x', area=-0.5*gx.area, duration=5e-3, system=sys)
gx_spoil = pp.make_trapezoid('x', area=1.5*gx.area, duration=2e-3, system=sys)

# Construct the sequence
seq = pp.Sequence()
for i in range(-n_phase//2, n_phase//2):
    # RF phase spoiling
    rf.phase_offset = (0.5 * (i**2+i+2) * 117) % 360 * np.pi / 180
    adc.phase_offset = rf.phase_offset
    seq.add_block(rf)
    # Phase encoding
    gy = pp.make_trapezoid('y', area=i / fov, duration=5e-3, system=sys)
    seq.add_block(gx_pre, gy)
    seq.add_block(adc, gx)
    # Rewind phase and spoil
    gy = pp.make_trapezoid('y', area=-i / fov, duration=5e-3, system=sys)
    seq.add_block(gx_spoil, gy)

    seq.add_block(pp.make_delay(1e-3))

In [ ]:
ok, error_report = seq.check_timing()
if ok:
    print("Timing check passed successfully")
else:
    print("Timing check failed:")
    [print(e, end="") for e in error_report]

seq.plot()

seq.set_definition("FOV", [fov, fov, slice_thickness])
seq.set_definition("Name", experiment_id)
seq.write(experiment_id + ".seq")

In [ ]:
phantom = mr0.VoxelGridPhantom.brainweb("subject05.npz")
phantom = phantom.interpolate(64, 64, 32).slices([16])
phantom.plot()
data = phantom.build()

In [ ]:
seq = mr0.Sequence.import_file(experiment_id + ".seq")
seq.plot_kspace_trajectory()

graph = mr0.compute_graph(seq, data, 200, 1e-3)
signal = mr0.execute_graph(graph, seq, data, print_progress=False)

In [ ]:
kspace = signal.view(n_phase, n_read)
reco = torch.fft.fftshift(torch.fft.fft2(torch.fft.fftshift(kspace)))

plt.figure()
plt.imshow(reco.abs(), origin="lower")
plt.show()